## Scraping antara news

In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
import re

In [11]:
jumlah_index = 600
threads_link = []
links = []
results = []
threads = []
keywords='ganjar'


In [12]:
def scrape_links(page_number,keywords):
    url = f"https://www.antaranews.com/search?q={keywords}&page={page_number}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article',{"class": "simple-post simple-big clearfix"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [13]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p,keywords)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 15 links from page 1
Scraped 15 links from page 8
Scraped 15 links from page 7
Scraped 15 links from page 3
Scraped 15 links from page 13
Scraped 15 links from page 10
Scraped 15 links from page 9
Scraped 15 links from page 4
Scraped 15 links from page 5
Scraped 15 links from page 17
Scraped 15 links from page 15
Scraped 15 links from page 50
Scraped 15 links from page 29
Scraped 15 links from page 30
Scraped 15 links from page 11
Scraped 15 links from page 6
Scraped 15 links from page 55
Scraped 15 links from page 20
Scraped 15 links from page 12
Scraped 15 links from page 18
Scraped 15 links from page 16
Scraped 15 links from page 31
Scraped 15 links from page 19
Scraped 15 links from page 75
Scraped 15 links from page 44
Scraped 15 links from page 58
Scraped 15 links from page 78
Scraped 15 links from page 51
Scraped 15 links from page 37
Scraped 15 links from page 28
Scraped 15 links from page 35
Scraped 15 links from page 43
Scraped 15 links from page 36
Scraped 15 links f

In [14]:
def scrape_url(url,keywords):
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
        }
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1',{"class": "post-title"})
            if title_elem:
                title_text = title_elem.text
            else:
                title_text = "Title not found"
            # Author berita
            # author_elem = soup.find('div', {"class": "tags-wrapper"})
            # if author_elem:
            #     author_text = author_elem.find('span')
            #     author_text = author_text.text.split(':')[1]
            # else:
            author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('span', {"class": "article-date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            # category_elements = soup.find('ul', {"class": "breadcrumb"})
            # if category_elements:
            #     category_text = category_elements.find_all('li')
            #     category_text= category_text[2].get_text()
            # else:
            category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "post-content clearfix"})
            if body_elem:
                 content_text = body_elem.text
            else:
                content_text ="Content not found"
            nama_berita_match = re.search(r'https://www\.(\w+)\.com/', url)
            if nama_berita_match:
                nama_berita = nama_berita_match.group(1)
            else:
                nama_berita = "Nama_berita not found"
            
            results.append({'title': title_text,
                            'keywords': keywords,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                             'nama_berita' : nama_berita,
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [15]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,keywords))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

Error fetching URL 'https://www.antaranews.com/berita/3766776/kaesang-psi-itu-warnanya-merah-putih-dan-hitam': HTTPSConnectionPool(host='www.antaranews.com', port=443): Max retries exceeded with url: /berita/3766776/kaesang-psi-itu-warnanya-merah-putih-dan-hitam (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f908c7179d0>: Failed to establish a new connection: [Errno 111] Connection refused'))
Error fetching URL 'https://www.antaranews.com/berita/3739605/pengamat-nilai-wacana-duet-ganjar-dan-prabowo-tak-realistis': HTTPSConnectionPool(host='www.antaranews.com', port=443): Max retries exceeded with url: /berita/3739605/pengamat-nilai-wacana-duet-ganjar-dan-prabowo-tak-realistis (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7f90fc356cd0>: Failed to establish a new connection: [Errno 111] Connection refused'))
Error fetching URL 'https://www.antaranews.com/berita/3740151/istri-ganjar-larang-anaknya-nikmati-fasilitas-sebagai-

In [16]:
df = pd.DataFrame(results)
print("hasil data scrapping",len(results))
df.head(100)

hasil data scrapping 502


,title,keywords,author,category,date,content,nama_berita,link
0,Aktivis 98 sepakat dukung Prabowo Subianto jad...,ganjar,Author not found,Category not found,"Rabu, 11 Oktober 2023 22:18 WIB",\n\t\t\t\tDenpasar (ANTARA) - Para aktivis 98 ...,antaranews,https://www.antaranews.com/berita/3769212/akti...
1,Andi Widjajanto resmi jadi Deputi Politik TPN ...,ganjar,Author not found,Category not found,"Rabu, 11 Oktober 2023 18:47 WIB","\n""Ada dua deputi yang pada hari ini akan saya...",antaranews,https://www.antaranews.com/berita/3768633/andi...
2,Ganjar ziarah ke makam KH Khoer Affandi pendir...,ganjar,Author not found,Category not found,"Selasa, 10 Oktober 2023 09:58 WIB",\nSaya memang sudah lama untuk bersilahturahmi...,antaranews,https://www.antaranews.com/berita/3765462/ganj...
3,Poltracking: Elektabilitas Prabowo di Jatim naik,ganjar,Author not found,Category not found,"Rabu, 11 Oktober 2023 15:53 WIB",\n\t\t\t\tSurabaya (ANTARA) - Hasil survei Pol...,antaranews,https://www.antaranews.com/berita/3768087/polt...
4,Gubernur Lemhannas akan pertimbangkan gabung T...,ganjar,Author not found,Category not found,"Rabu, 11 Oktober 2023 16:45 WIB",\n\t\t\t\tJakarta (ANTARA) - Gubernur Lembaga ...,antaranews,https://www.antaranews.com/berita/3768240/gube...
...,...,...,...,...,...,...,...,...
95,Andi Widjajanto pastikan posisinya di TPN Ganj...,ganjar,Author not found,Category not found,"Rabu, 11 Oktober 2023 20:04 WIB",\n\t\t\t\tJakarta (ANTARA) - Gubernur Lembaga ...,antaranews,https://www.antaranews.com/berita/3768873/andi...
96,OSO bertemu tokoh lintas agama bicara pesan ke...,ganjar,Author not found,Category not found,"Jumat, 6 Oktober 2023 22:30 WIB",\n\t\t\t\tManado (ANTARA) - Ketua Umum DPP Par...,antaranews,https://www.antaranews.com/berita/3761340/oso-...
97,Politkus PDIP sebut Ganjar punya simpanan pote...,ganjar,Author not found,Category not found,"Jumat, 6 Oktober 2023 22:31 WIB","\n""Kuncinya adalah semakin masif sosialisasi, ...",antaranews,https://www.antaranews.com/berita/3761343/poli...
98,Title not found,ganjar,Author not found,Category not found,Date not found,\nJakarta (ANTARA) - Mantan Gubernur Jawa Teng...,Nama_berita not found,https://otomotif.antaranews.com/berita/3751668...


In [17]:
filtered_df = df[df['content'].str.contains(keywords, case=False)|
                 df['title'].str.contains(keywords, case=False)]
print('hasil filter',len(filtered_df))
filtered_df.head(10)

hasil filter 496


,title,keywords,author,category,date,content,nama_berita,link
0,Aktivis 98 sepakat dukung Prabowo Subianto jad...,ganjar,Author not found,Category not found,"Rabu, 11 Oktober 2023 22:18 WIB",\n\t\t\t\tDenpasar (ANTARA) - Para aktivis 98 ...,antaranews,https://www.antaranews.com/berita/3769212/akti...
1,Andi Widjajanto resmi jadi Deputi Politik TPN ...,ganjar,Author not found,Category not found,"Rabu, 11 Oktober 2023 18:47 WIB","\n""Ada dua deputi yang pada hari ini akan saya...",antaranews,https://www.antaranews.com/berita/3768633/andi...
2,Ganjar ziarah ke makam KH Khoer Affandi pendir...,ganjar,Author not found,Category not found,"Selasa, 10 Oktober 2023 09:58 WIB",\nSaya memang sudah lama untuk bersilahturahmi...,antaranews,https://www.antaranews.com/berita/3765462/ganj...
3,Poltracking: Elektabilitas Prabowo di Jatim naik,ganjar,Author not found,Category not found,"Rabu, 11 Oktober 2023 15:53 WIB",\n\t\t\t\tSurabaya (ANTARA) - Hasil survei Pol...,antaranews,https://www.antaranews.com/berita/3768087/polt...
4,Gubernur Lemhannas akan pertimbangkan gabung T...,ganjar,Author not found,Category not found,"Rabu, 11 Oktober 2023 16:45 WIB",\n\t\t\t\tJakarta (ANTARA) - Gubernur Lembaga ...,antaranews,https://www.antaranews.com/berita/3768240/gube...
5,Kemampuan baca tulis Al Quran masyarakat Indon...,ganjar,Author not found,Category not found,"Rabu, 11 Oktober 2023 16:46 WIB",\n\t\t\t\tJakarta (ANTARA) - Kementerian Agama...,antaranews,https://www.antaranews.com/berita/3768246/kema...
6,Prabowo: KIM tunggu putusan MK umumkan pasanga...,ganjar,Author not found,Category not found,"Rabu, 11 Oktober 2023 18:16 WIB",\nSaya katakan ini keputusan dengan semua part...,antaranews,https://www.antaranews.com/berita/3768519/prab...
7,Mahasiswa Unand ciptakan inovasi pendeteksi ka...,ganjar,Author not found,Category not found,"Senin, 9 Oktober 2023 16:35 WIB",\n\t\t\t\tPadang (ANTARA) - Lima mahasiswa lin...,antaranews,https://www.antaranews.com/berita/3764163/maha...
8,Projo titip 11 program rakyat ke kandidat capr...,ganjar,Author not found,Category not found,"Kamis, 12 Oktober 2023 15:37 WIB",\n\t\t\t\tJakarta (ANTARA) - Organisasi Masyar...,antaranews,https://www.antaranews.com/berita/3770058/proj...
9,Projo akan undang bacapres dalam Rakernas pert...,ganjar,Author not found,Category not found,"Senin, 9 Oktober 2023 16:40 WIB","\n""Kita undang bacapres,""Jakarta (ANTARA) - Ke...",antaranews,https://www.antaranews.com/berita/3764181/proj...


Tempat Simpan Save

In [18]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'../tempat_simpan_nasional/antaranews_{keywords}_{current_datetime}.xlsx'
filtered_df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')

Data has been saved to ../tempat_simpan_nasional/antaranews_ganjar_2023-10-12_16-54-43.xlsx
